# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
from dotenv import load_dotenv
import os
from glob import glob
import dask.dataframe as dd

# Load .env variables into os.environ
load_dotenv()

True

In [5]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
print("PRICE_DATA =", PRICE_DATA)

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
print("Parquet files found:", len(parquet_files))
print("Example parquet files:", parquet_files[:5])

PRICE_DATA = ../../05_src/data/prices/
Parquet files found: 2705
Example parquet files: ['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
import pandas as pd

dd_px = dd.read_parquet(parquet_files)

print(dd_px.columns)
dd_px.head()

def add_features(pdf: pd.DataFrame) -> pd.DataFrame:
    pdf = pdf.sort_values("Date")
    
    pdf["Close_lag_1"] = pdf["Close"].shift(1)
    pdf["Adj_Close_lag_1"] = pdf["Adj Close"].shift(1)  # keeps consistent with Notebook 2
    pdf["returns"] = (pdf["Close"] / pdf["Close_lag_1"]) - 1
    pdf["hi_lo_range"] = pdf["High"] - pdf["Low"]
    
    return pdf

# Meta tells Dask what the output columns will be
meta = dd_px._meta.copy()
meta["Close_lag_1"] = meta["Close"]
meta["Adj_Close_lag_1"] = meta["Adj Close"]
meta["returns"] = meta["Close"]
meta["hi_lo_range"] = meta["High"]

dd_feat = (
    dd_px
    .groupby("ticker", group_keys=False)
    .apply(add_features, meta=meta)
)

dd_feat.head()

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year'],
      dtype='object')


c:\Users\hmwan\dsi\production\production-env\Lib\site-packages\dask\dataframe\groupby.py:116: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range


c:\Users\hmwan\dsi\production\production-env\Lib\site-packages\dask\dataframe\groupby.py:116: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
c:\Users\hmwan\dsi\production\production-env\Lib\site-packages\dask\dataframe\groupby.py:116: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(func, *args, **kwargs)
c:\Users\hmwan\dsi\production\production-env\Lib\site-packages\d

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [10]:
feat_df = dd_feat.compute()
feat_df.head()

feat_df = feat_df.sort_values(["ticker", "Date"])

feat_df["ma_return_10"] = (
    feat_df.groupby("ticker")["returns"]
    .rolling(10)
    .mean()
    .reset_index(level=0, drop=True)
)

feat_df[["ticker", "Date", "returns", "ma_return_10"]].head(15)

,ticker,Date,returns,ma_return_10
137315,ACN,2001-07-19,NaN,NaN
137316,ACN,2001-07-20,-0.010547,NaN
137317,ACN,2001-07-23,-0.000666,NaN
137318,ACN,2001-07-24,-0.009333,NaN
137319,ACN,2001-07-25,0.006057,NaN
137320,ACN,2001-07-26,-0.030100,NaN
137321,ACN,2001-07-27,0.000690,NaN
137322,ACN,2001-07-30,0.013094,NaN
137323,ACN,2001-07-31,0.017687,NaN
137324,ACN,2001-08-01,0.036096,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Was it necessary to convert to pandas to calculate the moving average return?

No — it wasn’t strictly necessary. Dask supports rolling operations, and rolling means can be computed in Dask as well.

Would it have been better to do it in Dask? Why?

If the dataset is large, yes.
Doing it in Dask would be better because Dask can process data in parallel and avoids loading everything into memory.
However, converting to pandas is often easier to implement and debug if the data fits in RAM.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.